## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [6]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Avarua,CK,-21.21,-159.78,75.20,78,86,17.87,overcast clouds
1,1,Cape Town,ZA,-33.93,18.42,66.99,88,75,9.17,light rain
2,2,Tuktoyaktuk,CA,69.45,-133.04,6.80,77,90,13.87,light snow
3,3,Jamestown,US,42.10,-79.24,64.00,42,1,11.41,clear sky
4,4,Ushuaia,AR,-54.80,-68.30,50.00,46,75,13.87,broken clouds


In [8]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [10]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] >= min_temp) & (city_data_df['Max Temp'] <= max_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Avarua,CK,-21.21,-159.78,75.20,78,86,17.87,overcast clouds
7,7,Caucaia,BR,-3.74,-38.65,84.20,70,20,12.75,few clouds
16,16,Kapaa,US,22.08,-159.32,80.01,94,40,4.70,moderate rain
20,20,Tual,ID,-5.67,132.75,82.27,78,64,11.27,broken clouds
22,22,Alyangula,AU,-13.85,136.42,78.80,88,29,1.12,scattered clouds


In [25]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                196
City                   196
Country                194
Lat                    196
Lng                    196
Max Temp               196
Humidity               196
Cloudiness             196
Wind Speed             196
Current Description    196
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

In [34]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Avarua,CK,75.20,overcast clouds,-21.21,-159.78,
7,Caucaia,BR,84.20,few clouds,-3.74,-38.65,
16,Kapaa,US,80.01,moderate rain,22.08,-159.32,
20,Tual,ID,82.27,broken clouds,-5.67,132.75,
22,Alyangula,AU,78.80,scattered clouds,-13.85,136.42,


In [81]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json' 
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url,params=params).json()

    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        continue 

In [82]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Avarua,CK,75.20,overcast clouds,-21.21,-159.78,Paradise Inn
7,Caucaia,BR,84.20,few clouds,-3.74,-38.65,M.O.S. Logística - Filial Fortaleza
16,Kapaa,US,80.01,moderate rain,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
20,Tual,ID,82.27,broken clouds,-5.67,132.75,Grand Vilia Hotel
22,Alyangula,AU,78.80,scattered clouds,-13.85,136.42,Groote Eylandt Lodge


In [87]:
# 7. Drop the rows where there is no Hotel Name.
len(hotel_df[hotel_df['Hotel Name'] == ''])
drop_list = hotel_df[hotel_df['Hotel Name']==''].index
hotel_df.drop(index=drop_list,inplace=True)

17

In [88]:
# 8a. Create the output File (CSV)
# 8b. Export the City_Data into a csv
hotel_df.to_csv('WeatherPy_vacation.csv', index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Weather} and {Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure
